In [27]:
from gensim import corpora, models, similarities
from gensim.models import Word2Vec,keyedvectors 
from gensim.models.word2vec import LineSentence
from gensim.corpora import Dictionary
import jieba.posseg as jp, jieba
import pandas as pd
import numpy as np
from string import punctuation
import re

jieba.enable_paddle()

add_punc='，。、【 】 “”：；（）《》‘’{}？！⑦()、%^>℃：.”“^-——=&#@￥'
all_punc=punctuation+add_punc

Paddle enabled successfully......


In [8]:
model_path='./model/baike_26g_news_13g_novel_229g.model'
job_path='./model/职业分类大典.xlsx'
job_post_path='./model/招聘20210611.xlsx'
test_input="前台行政"

In [4]:
model=Word2Vec.load(model_path)
word_vectors = model.wv

In [66]:
#jieba精准分词
def jieba_cut(sentence, word_vectors):
    word_list = []
    seg_list = jieba.cut_for_search(sentence)
    for word in seg_list:
        if word != '' and word is not None and word not in all_punc and word in word_vectors.key_to_index:
            word_list.append(word)
    return word_list


In [90]:
# Load job list
data = pd.read_excel(job_path, keep_default_na=False)
job_dictionary = pd.DataFrame(data, columns= ['小类'])
job_dictionary.replace('', np.nan, inplace=True)
job_dictionary.dropna(inplace=True)
job_dictionary.drop_duplicates(keep='first',inplace=True)
job_categories = job_dictionary.values.tolist()

In [85]:
def find_job_category(job_name, job_categories):
    job_scores=dict()   
    for job in job_categories:
        for title in job:
            title_list = jieba_cut(title, word_vectors)
            job_name_list = jieba_cut(job_name, word_vectors)
            if title_list and job_name_list:
                job_scores[title]=word_vectors.n_similarity(title_list,job_name_list) 

    job_scores = sorted(job_scores.items(),key=lambda d:d[1],reverse=True)
    # Return top job
    return job_scores

In [37]:
# keep chinese only
def keep_all_chinese(file):
    pattern = re.compile(r'[^\u4e00-\u9fa5]')
    chinese=re.sub(pattern,'',file)
    return chinese

In [38]:
# read test jobs
posted_jobs_data = pd.read_excel(job_post_path, keep_default_na=False)
posted_jobs = pd.DataFrame(posted_jobs_data, columns= ['职位名称']).values.tolist()

In [75]:
# find job category list
#job_list=[]
#for job in job_categories:
#    for title in job:
#        job_list.append(jieba_cut(title,word_vectors))

In [91]:
#f = open("result.txt", "a")

for post_job in posted_jobs:
    post_job_chinese=keep_all_chinese(post_job[0])
    if post_job_chinese:
        job_scores = find_job_category(post_job_chinese, job_categories)
        print('post_job: {} category&scores: {}'.format(post_job, job_scores[0]))

#f.close()

post_job: ['报告厅文员 (MJ000752)'] category&scores: ('体育工作人员', 0.58045805)
post_job: ['建筑设计主管/经理'] category&scores: ('物业管理员', 0.68842936)
post_job: ['前台行政'] category&scores: ('其他行政办公人员', 0.63815093)
post_job: ['西餐厅前台预定礼仪接待'] category&scores: ('餐厅服务人员', 0.7746835)
post_job: ['文员'] category&scores: ('会计人员', 0.6372721)
post_job: ['链家储备经理 7K+ 五险一金'] category&scores: ('粮油管理人员', 0.45405933)
post_job: ['消防中控员'] category&scores: ('供用电人员', 0.44368938)
post_job: ['会议弱电工'] category&scores: ('行政事务人员', 0.42701167)
post_job: ['办公室文员,销售助理,销售文员'] category&scores: ('饭店服务部人员', 0.71644795)
post_job: ['客服+无责底薪3k5+长白班'] category&scores: ('卫生单位负责人', 0.5533663)
post_job: ['石楼石基市桥置业顾问'] category&scores: ('饭店服务部人员', 0.47621116)
post_job: ['五险一金诚聘工厂保安员'] category&scores: ('饭店服务部人员', 0.61486846)
post_job: ['助理文员'] category&scores: ('律师', 0.69781464)
post_job: ['咨询经理'] category&scores: ('律师', 0.6890873)
post_job: ['保安（高薪，五险一金）'] category&scores: ('人民警察', 0.4947936)
post_job: ['龙虾自由/5000底薪/租房管家'] category&scores: ('饭店

IndexError: list index out of range

In [ ]:
print(job_categories)